### Introduction

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth.

First of, let's connect our Jupyter Notebook to our database file:

In [84]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

### Overview of the Data

We need an information about the data in the database. We'll begin by getting a sense of what the data looks like.


In [85]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


There are two table in database. Let's look the first five rows of the facts table in the database.

In [86]:
%%sql
SELECT *
  FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- **name** — the name of the country.
- **area** — the country's total area (both land and water).
- **area_land** — the country's land area in square kilometers.
- **area_water** — the country's waterarea in square kilometers.
- **population** — the country's population.
- **population_growth** — the country's population growth as a percentage.
- **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.
- **death_rate** — the country's death rate, or the number of death per year per 1,000 people.


Let's start by calculating some summary statistics and look for any outlier values.

In [87]:
%%sql
SELECT MIN(population) AS minimum_population,
       MAX(population) AS maximum_population,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;



Done.


minimum_population,maximum_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


It is interesting things that there's a country which has 0 population and there's a country which has more than 7.2 billion people. These are outliers. We should check them out.

In [88]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


This country is Antarctica, which has the population of 0. This seems to match the information on CIA Factbook page:

https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society

![Population of Antarctica on CIA Factbook page](https://app.dataquest.io/jupyter/files/notebook/antarctica.png)

In [89]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.


### Summary Statistics Revisited

If we recalculate the summary statistics while excluding the row for the whole world:

In [90]:
%%sql
SELECT MIN(population) AS minimum_population,
       MAX(population) AS maximum_population,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts
 WHERE name <> 'World';

Done.


minimum_population,maximum_population,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

In [91]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                        WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


This is correct that China's population is about 1.4 billion.

### Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.


In [92]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


We understand that the average population is around 32 million and the average area is 555 thousand square kilometers.

### Densely Populated Countries

Which countries are densely populated? For this query, we'll identify countries that have:

- Above average values for population.
- Below average values for area.

In [93]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
              );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are generally known to be densely populated, so we have confidence in our results!

#### Which country has the most people? 

In [94]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                      WHERE name <> 'World'
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


Chine has the most people.

#### Which country has the highest growth rate?

In [95]:
%%sql
SELECT *
  FROM facts
 WHERE population_growth = (SELECT MAX(population_growth)
                              FROM facts
                             WHERE name <> 'World'
                            );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan has the highest growth rate.

#### Which countries have the highest ratios of water to land? 

In [96]:
%%sql
SELECT name, area, area_land, area_water, CAST(area_water as FLOAT)/area_land AS ratio
  FROM facts
    ORDER BY ratio DESC
    LIMIT 10;

Done.


name,area,area_land,area_water,ratio
British Indian Ocean Territory,54400,60,54340,905.6666666666666
Virgin Islands,1910,346,1564,4.520231213872832
Puerto Rico,13791,8870,4921,0.5547914317925592
"Bahamas, The",13880,10010,3870,0.3866133866133866
Guinea-Bissau,36125,28120,8005,0.2846728307254623
Malawi,118484,94080,24404,0.25939625850340137
Netherlands,41543,33893,7650,0.22571032366565366
Uganda,241038,197100,43938,0.22292237442922375
Eritrea,117600,101000,16600,0.16435643564356436
Liberia,111369,96320,15049,0.15623961794019933


British Indian Ocean Territory has the most water/land ratio but this is outlier. This country is very small island in Indian Ocean.

![British Indian Ocean Territory](https://app.dataquest.io/jupyter/files/notebook/british_indian_ocean_territory.png)

The second ratio is belong to Virgin Islands.

In [97]:
%%sql
SELECT *, CAST(area_water as FLOAT)/area_land AS ratio
  FROM facts
  WHERE ratio = (SELECT MAX(CAST(area_water as FLOAT)/area_land)
                    FROM facts
                 WHERE name <> 'British Indian Ocean Territory'
                    );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,ratio
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.520231213872832


#### Which countries have more water than land?

In [98]:
%%sql
SELECT name, CAST(area_water as FLOAT)/area_land AS ratio
  FROM facts
  WHERE ratio > (SELECT AVG(CAST(area_water as FLOAT)/area_land)
                    FROM facts
                 WHERE name <> 'British Indian Ocean Territory'
                    );

Done.


name,ratio
Albania,0.04927366961092051
Armenia,0.054604120129064286
Azerbaijan,0.048058187803313605
"Bahamas, The",0.3866133866133866
Bangladesh,0.1405085657217485
Brunei,0.0949667616334283
Burundi,0.0837227414330218
Canada,0.09799992456155804
Colombia,0.09647636468662751
Eritrea,0.16435643564356436


#### Which countries have more water than land?

In [99]:
%%sql
SELECT *
  FROM facts
 WHERE area_water > area_land;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


#### Which countries will add the most people to their populations next year?

We can find that which country has higest population growth.

In [100]:
%%sql
SELECT *
  FROM facts
 WHERE population_growth = (SELECT MAX(population_growth)
                        FROM facts
                        WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


#### Which countries have a higher death rate than birth rate?

In [101]:
%%sql
SELECT *
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY death_rate;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
214,sb,Saint Pierre and Miquelon,242,242,0,5657,1.08,7.42,9.72,8.49
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
140,po,Portugal,92090,91470,620,10825309,0.09,9.27,11.02,2.67
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32


#### Which countries have the highest population/area ratio?

In [102]:
%%sql
SELECT name, population, area, CAST(population as FLOAT)/area AS ratio
  FROM facts
    ORDER BY ratio DESC
    LIMIT 10;

Done.


name,population,area,ratio
Macau,592731,28,21168.964285714286
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.279770444763
Hong Kong,7141106,1108,6445.041516245487
Gaza Strip,1869055,360,5191.819444444444
Gibraltar,29258,6,4876.333333333333
Bahrain,1346613,760,1771.8592105263158
Maldives,393253,298,1319.6409395973155
Malta,413965,316,1310.01582278481
Bermuda,70196,54,1299.9259259259259


We can compare this list with densely populated countries list. Densely populated countries list was created by using average population and average area values above. It was depended on average values of countries. But this new table shows exact amount of population/area ratio owned. With this table, we can understand how many people live per kilometer in that country. 